<a href="https://colab.research.google.com/github/aureavaleria/DataBalancing-Research/blob/main/papers/Artigo%201/V3/Vers%C3%A3o_3_(ajustes_de_hiperpar%C3%A2metros_XGBoost).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Machine learning for predicting liver and/or lung metastasis in colorectal cancer: A retrospective study based on the SEER database***

Este notebook apresenta um processo de otimização de hiperparâmetros para o modelo XGBoost, utilizando dados extraídos da base SEER, amplamente utilizada em pesquisas oncológicas. O objetivo é ajustar os parâmetros do modelo de forma iterativa, maximizando sua performance com base na métrica AUC-ROC.


### Parte 1:  Importação das Bibliotecas e Carregamento do Dataset

Nesta etapa, importamos as bibliotecas necessárias para análise e carregamos o dataset. Realizamos uma verificação inicial para identificar e remover valores faltantes e definimos as variáveis preditoras (X) e as variáveis alvo (y), preparando os dados para o pré-processamento e a modelagem.

In [1]:
# Importação das bibliotecas essenciais para análise de dados
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier

# Carregar o dataset
df = pd.read_csv('https://raw.githubusercontent.com/aureavaleria/Reprodu-o/main/export.csv')

# Verificar se existem valores faltantes
print("Valores faltantes por coluna:\n", df.isnull().sum())

# Remover linhas com valores faltantes
df.dropna(inplace=True)

Valores faltantes por coluna:
 Patient ID                                         0
Age recode with <1 year olds                       0
Sex                                                0
Race recode (White, Black, Other)                  0
Histologic Type ICD-O-3                            0
Grade Recode (thru 2017)                           0
Primary Site                                       0
Derived AJCC T, 7th ed (2010-2015)                 0
Derived AJCC N, 7th ed (2010-2015)                 0
CS tumor size (2004-2015)                          0
CEA Pretreatment Interpretation Recode (2010+)     0
Tumor Deposits Recode (2010+)                      0
Marital status at diagnosis                        0
Origin recode NHIA (Hispanic, Non-Hisp)            0
SEER Combined Mets at DX-lung (2010+)             15
SEER Combined Mets at DX-liver (2010+)            12
SEER Combined Mets at DX-bone (2010+)             14
ICD-O-3 Hist/behav                                 0
ICD-O-3 Hist/be

## Parte 2: Preparação das Variáveis Alvo e Codificação de Variáveis Categóricas
Nesta etapa, preparamos as variáveis alvo (y), combinando as informações de metástase hepática e pulmonar em uma coluna binária para indicar a presença de metástase. Também aplicamos LabelEncoder para transformar variáveis categóricas de X em valores numéricos, facilitando o uso dos dados em modelos de aprendizado de máquina.

In [2]:
#Definir as variáveis preditoras (X)
X = df[['Age recode with <1 year olds', 'Sex', 'Race recode (White, Black, Other)',
        'Histologic Type ICD-O-3', 'Grade Recode (thru 2017)', 'Primary Site',
        'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)',
        'CS tumor size (2004-2015)', 'CEA Pretreatment Interpretation Recode (2010+)',
        'Tumor Deposits Recode (2010+)', 'Marital status at diagnosis',
        'Origin recode NHIA (Hispanic, Non-Hisp)']]

# Definir a variável alvo
y = df[['SEER Combined Mets at DX-liver (2010+)', 'SEER Combined Mets at DX-lung (2010+)']]

# Aplicar LabelEncoder às colunas categóricas em X
for col in X.columns:
    if X[col].dtype == 'object':  # Verifica se a coluna é categórica
        X[col] = LabelEncoder().fit_transform(X[col])

# Criar coluna binária 'Binary Mets'
y['Binary Mets'] = np.where((y['SEER Combined Mets at DX-liver (2010+)'] == 'Yes') |
                            (y['SEER Combined Mets at DX-lung (2010+)'] == 'Yes'), 1, 0)

# Verificar os tamanhos de X e y
print(f"Tamanho de X: {len(X)}")
print(f"Tamanho de y: {len(y)}")

# Renomear colunas de X para remover caracteres inválidos
X.columns = [str(col).replace('[', '').replace(']', '').replace('<', '').replace('>', '').replace(' ', '_')
             for col in X.columns]

<ipython-input-2-b9e028aac685>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-2-b9e028aac685>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-2-b9e028aac685>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

Tamanho de X: 53448
Tamanho de y: 53448


<ipython-input-2-b9e028aac685>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-2-b9e028aac685>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-2-b9e028aac685>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

##3. Otimização de hiperparâmetros

In [3]:
# Configurar modelo XGBoost e hiperparâmetros para GridSearchCV
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 6, 10],
    "learning_rate": [0.01, 0.1, 0.2],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0],
    "reg_alpha": [0.01, 0.1, 1.0],
    "reg_lambda": [0.1, 1.0, 10.0]
}

# GridSearchCV para XGBoost
print("\nBuscando os melhores parâmetros para XGBoost...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring="roc_auc", cv=cv, n_jobs=-1, verbose=1)
grid_search.fit(X, y['Binary Mets'])

# Exibir os melhores parâmetros e pontuação
print("\nMelhores parâmetros para XGBoost:", grid_search.best_params_)
print(f"Melhor pontuação (AUC-ROC): {grid_search.best_score_:.4f}")

# Salvar resultados em arquivo
with open("melhores_hiperparametros_xgb.txt", "w") as file:
    file.write("Modelo: XGBoost\n")
    file.write("Melhores Parâmetros:\n")
    for param, value in grid_search.best_params_.items():
        file.write(f"  {param}: {value}\n")
    file.write(f"Melhor Pontuação (AUC-ROC): {grid_search.best_score_:.4f}\n")
    file.write("-" * 50 + "\n")

print("Resultados salvos no arquivo 'melhores_hiperparametros_xgb.txt'.")



Buscando os melhores parâmetros para XGBoost...
Fitting 5 folds for each of 2187 candidates, totalling 10935 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:40:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Melhores parâmetros para XGBoost: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 10.0, 'subsample': 1.0}
Melhor pontuação (AUC-ROC): 0.8885
Resultados salvos no arquivo 'melhores_hiperparametros_xgb.txt'.
